In [1]:
import pandas as pd
import zipfile
import os

In [2]:
load_df = pd.DataFrame(columns = ['Name','Load','Date']) 

In [3]:
def compile_data(name):
    # Reading the file
    df = pd.read_csv(f'data/Load Data/{name}') 
    # Extracting the date
    df['Date'] = pd.to_datetime(df['Time Stamp']).dt.date 
    # Droping the unnecessary columns
    df.drop(['Time Stamp','Time Zone','PTID'], axis = 1,inplace = True) 
    # Grouping the data to find avg load required and converting it to dataframe
    gr_df = df.groupby('Name')['Load'].mean().reset_index() 
    gr_df['Date'] = df['Date']
     # Concatenating with the global load_df
    global load_df
    load_df = pd.concat([load_df, gr_df], ignore_index=True)
    return load_df

In [4]:
def unzip_folder(zip_path,extract_to):
    # Open the zip file and extract contents
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Contents extracted to {extract_to}")

*All the data of load is taken from [New York ISO Independent Systems Operator](https://www.nyiso.com/)*

```python
# Define the base path
base_path = 'data/Load Data'

# Iterate through the range of years
for i in range(2008, 2025):
    # Construct the directory path
    directory_path = os.path.join(base_path, str(i))
    
    # Check if the directory exists
    if os.path.exists(directory_path):
        # Iterate through the files in the directory
        for zip_file in os.listdir(directory_path):
            # Construct the full path to the zip file
            zip_path = os.path.join(directory_path, zip_file)
            
            # Call your unzip function
            unzip_folder(zip_path, base_path)
    else:
        print(f"Directory for year {i} does not exist.")

#### Above code was used to extract the File which where in zip format

In [6]:
# Define the base path
base_path = 'data/Load Data'

# Extracting the names of all the .csv files in Load Data folder
file_names = set([i for i in os.listdir(base_path) if i.endswith('.csv')])

In [7]:
# Extracting and compiling the data from all .csv files with the help of  compile_data function
for name in file_names:
    compile_data(name) 

C:\Users\hasee\AppData\Local\Temp\ipykernel_44620\3631843620.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  load_df = pd.concat([load_df, gr_df], ignore_index=True)


In [8]:
load_df.to_csv('data/Newyork_state_load_data.csv')

In [9]:
load_df.shape

(92276, 3)